In [1]:
import pandas as pd
import numpy as np
from math import log2
from collections import Counter
import glob
import os

In [2]:
# Define paths
path = "/Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze" 
save_matrix_path = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Matrices/"
entropy_log_path = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/entropy_log.csv"

# Ensure directories exist
os.makedirs(save_matrix_path, exist_ok=True)

# Load all CSV files in the path
files = glob.glob(path + "/*.csv")

# Mapping dictionary
mapping = {
    "TaskBuilding_Public": "Task_Building",
    "TaskBuilding_Residential": "Task_Building",
    "Active_Agent": "Agent_Body",
    "Passive_Agent": "Agent_Body",
    "Active_Agent_Face": "Agent_Face",
    "Passive_Agent_Face": "Agent_Face"
}



# Initialize list to store entropy results
entropy_results = []
failed_files = []

# Function to calculate conditional entropy H(X|Y=y) for each row
def calculate_transition_entropy(matrix, stationary_distribution):
    total_entropy = 0
    category_entropies = {}
    for i, row in matrix.iterrows():
        row_entropy = 0
        for p in row:
            if p > 0:
                row_entropy -= p * log2(p)
        category_entropies[i] = row_entropy
        # Weight each row entropy by its stationary probability
        total_entropy += row_entropy * stationary_distribution[i]
    return total_entropy, category_entropies

# Loop through all files
for filename in files:
    try:
        print(f"Processing file: {filename}")  # Debugging statement

        # Read participant data
        One_participant = pd.read_csv(filename)
        if 'timeStampDataPointEnd' not in One_participant.columns or 'Collider_CategoricalN' not in One_participant.columns:
            print(f"Skipping {filename} due to missing columns.")
            failed_files.append({'filename': filename[-10:-4], 'error': 'Missing required columns'})
            continue

        # Convert timestamps (if required)
        try:
            One_participant['date_seconds'] = pd.to_datetime(One_participant['timeStampDataPointEnd'], unit='s')
        except Exception as e:
            print(f"Timestamp conversion error in {filename}: {e}")
            failed_files.append({'filename': filename[-10:-4], 'error': str(e)})
            continue

        # Filter for the desired gaze events
        data_Reduced = One_participant[One_participant['events'] == -2]
        
        # Check if data_Reduced is empty to avoid errors in transition matrix creation
        if data_Reduced.empty:
            print(f"No valid events for {filename[-10:-4]}, skipping.")
            failed_files.append({'filename': filename[-10:-4], 'error': 'No valid events in filtered data'})
            continue

        # Get gaze sequence for category transitions
        # Apply mapping
        #data_Reduced = data_Reduced.copy()
        #data_Reduced.loc[:, "Mapped_Column_Collider_Categorical"] = data_Reduced["Collider_CategoricalN"].replace(mapping)
        gaze_sequence = data_Reduced['Collider_CategoricalN'].reset_index(drop=True)

        # Create transition matrix
        categories = gaze_sequence.unique()
        transition_matrix = pd.DataFrame(0, index=categories, columns=categories, dtype=float)

        # Build the transition matrix by counting transitions
        for i in range(len(gaze_sequence) - 1):
            current_category = gaze_sequence.iloc[i]
            next_category = gaze_sequence.iloc[i + 1]
            transition_matrix.loc[current_category, next_category] += 1

        # Normalize the transition matrix so each row sums to 1 (row normalization only)
        transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0).fillna(0)

        # Save the normalized transition matrix
        matrix_filename = os.path.join(save_matrix_path, f"{filename[-10:-4]}_transition_matrix.csv")
        transition_matrix.to_csv(matrix_filename, index=True)
        print(f"Saved transition matrix for {filename[-10:-4]}")  # Debugging statement

        # Calculate the stationary distribution (normalized left eigenvector of eigenvalue 1)
        eigvals, eigvecs = np.linalg.eig(transition_matrix.T)
        stationary_distribution = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
        stationary_distribution /= stationary_distribution.sum()

        # Ensure stationary distribution matches the order of `categories`
        stationary_distribution_dict = {categories[i]: stationary_distribution[i] for i in range(len(categories))}

        # Calculate overall transition entropy and per-category entropies using stationary distribution
        overall_transition_entropy, transition_entropy_per_category = calculate_transition_entropy(transition_matrix, stationary_distribution_dict)

        # Calculate stationary entropy per category (based on stationary distribution values)
        stationary_entropy_per_category = {
            category: (-stationary_distribution_dict[category] * log2(stationary_distribution_dict[category])).item() if stationary_distribution_dict[category] > 0 else 0
            for category in categories
        }

        # Normalize entropies by the number of categories
        num_categories = len(transition_matrix)
        normalized_overall_transition_entropy = overall_transition_entropy / log2(num_categories) if num_categories > 1 else 0

        # Add entropy values to results
        result = {
            'Filename': filename[-10:-4],
            'Overall_Transition_Entropy': normalized_overall_transition_entropy
        }

        # Add each category's transition and stationary entropy to the result, ensuring scalar values
        for category in categories:
            result[f'Transition_Entropy_{category}'] = transition_entropy_per_category.get(category, 0) / log2(num_categories) if num_categories > 1 else 0
            result[f'Stationary_Entropy_{category}'] = stationary_entropy_per_category.get(category, 0) / log2(num_categories) if num_categories > 1 else 0

        entropy_results.append(result)

    except Exception as e:
        # Log any failed files
        print(f"Error processing {filename}: {e}")  # Debugging statement
        failed_files.append({'filename': filename[-10:-4], 'error': str(e)})

# Save entropy results to a CSV
entropy_df = pd.DataFrame(entropy_results)
entropy_df.to_csv(entropy_log_path, index=False)

# Log any failed files if they exist
if failed_files:
    failed_df = pd.DataFrame(failed_files)
    failed_df.to_csv("/Volumes/TwoTeras/1_Experiment_2/Proxemics/failed_files_log.csv", index=False)
    print(f"Failures logged for {len(failed_files)} files.")
else:
    print("No failures detected.")


Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_1.csv
Saved transition matrix for 1031_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_2.csv
Saved transition matrix for 1031_2
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_3.csv
Saved transition matrix for 1031_3
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_4.csv
Saved transition matrix for 1031_4
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_5.csv
Saved transition matrix for 1031_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1268_1.csv
Saved transition matrix for 1268_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1268_2.csv
Saved transition matrix for 1268_2
Processing file: /Volumes/T

Saved transition matrix for 5161_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_1.csv
Saved transition matrix for 5189_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_2.csv
Saved transition matrix for 5189_2
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_3.csv
Saved transition matrix for 5189_3
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_4.csv
Saved transition matrix for 5189_4
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_5.csv
Saved transition matrix for 5189_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5743_1.csv
Saved transition matrix for 5743_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5743_2.csv
Saved transition matrix for

Saved transition matrix for 9297_4
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9297_5.csv
Saved transition matrix for 9297_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_1.csv
Saved transition matrix for 9627_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_2.csv
Saved transition matrix for 9627_2
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_3.csv
Saved transition matrix for 9627_3
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_4.csv
Saved transition matrix for 9627_4
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_5.csv
Saved transition matrix for 9627_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1142_1.csv
Saved transition matrix for

In [3]:
data = pd.read_csv("/Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_1.csv")
data['date_seconds'] = pd.to_datetime(data['timeStampDataPointEnd'], unit='s')      

In [4]:
data.names.unique()

array(['pavement_O.002', 'terrain_O.001', 'Wall', 'building01_LOD1',
       'TaskBuilding_27', 'Building_100', 'Building_97',
       'maraz_cafe_collider', 'barbwire0', 'CollisionObject1',
       'Building_161', 'TaskBuilding_35', '23_Cma', 'Graffity_35',
       'Building_166', 'Fence_5', 'road_base_network.004',
       'pavement_Vb.003', 'Building_215', 'Crane_59', 'cyclone0',
       'crane_1', 'Fence_34', 'Building_162', 'Building_99', '20_Cma',
       'Complete_fence.002', 'CollisionObject2', 'Building_94',
       'Building_157', 'building01_LOD0', 'CollisionObject0',
       'terrain_E.001', 'Wall_9', 'Fence_9', 'Fence_10', 'Building_98',
       'Maraz_cafe_place.001', 'Building_148', 'TaskBuilding_5', '05_Cma',
       'pileOfClay_LOD0', 'Building_101', 'Wall_3', 'Body', 'Fence_12',
       'Hedge_9', 'Building_102', 'Lamppost_v1 (15)', 'pavement_E.008',
       'Cypress_v1_2 (1)', 'TaskBuilding_30', 'Graffity_30',
       'Lamppost_v1 (56)', 'Lamppost_v1 (11)', 'road.009', 'Building_1

In [11]:
data_Reduced = data[data['events'] == 2]

In [12]:
data_Reduced[['date_seconds','names', 'Collider_CategoricalN']].head(30)

date_seconds                      names  \
1   2023-01-18 12:51:23.249797120      road_base_network.004   
12  2023-01-18 12:51:23.404053248               Building_161   
30  2023-01-18 12:51:23.626261760               Building_161   
35  2023-01-18 12:51:23.715045632  Concrete_fence_v1_s2 (35)   
41  2023-01-18 12:51:23.793412864  Concrete_fence_v1_s2 (36)   
61  2023-01-18 12:51:24.093494784              terrain_S.001   
81  2023-01-18 12:51:24.393572864              terrain_S.001   
104 2023-01-18 12:51:24.803268352              terrain_O.001   
122 2023-01-18 12:51:25.059700736              terrain_O.001   
131 2023-01-18 12:51:25.214950912              terrain_O.001   
135 2023-01-18 12:51:25.326052352              terrain_O.001   
144 2023-01-18 12:51:25.459476224              terrain_O.001   
166 2023-01-18 12:51:25.748644864            TaskBuilding_35   
171 2023-01-18 12:51:25.858756864            TaskBuilding_35   
178 2023-01-18 12:51:25.969364992            TaskBuilding_35   
186 2023-01-18 12:51:26.103284480                     23_Cma   
195 2023-01-18 12:51:26.236708352                     23_Cma   
200 2023-01-18 12:51:26.314086144                     23_Cma   
214 2023-01-18 12:51:26.569525760                     23_Cma   
221 2023-01-18 12:51:26.713861120                     23_Cma   
242 2023-01-18 12:51:27.025349120                     23_Cma   
250 2023-01-18 12:51:27.169189632                     23_Cma   
262 2023-01-18 12:51:27.347252992                      Wall6   
277 2023-01-18 12:51:27.580372736               Building_166   
294 2023-01-18 12:51:27.814486016               Building_166   
300 2023-01-18 12:51:27.925588480               Building_166   
329 2023-01-18 12:51:28.347684608                    Fence_5   
338 2023-01-18 12:51:28.502436608               Building_166   
351 2023-01-18 12:51:28.680005632               Building_166   
356 2023-01-18 12:51:28.746964224               Building_166   

        Collider_CategoricalN  
1                  Background  
12                   Building  
30                   Building  
35                 Background  
41                 Background  
61                 Background  
81                 Background  
104                Background  
122                Background  
131                Background  
135                Background  
144                Background  
166  TaskBuilding_Residential  
171  TaskBuilding_Residential  
178  TaskBuilding_Residential  
186              Active_Agent  
195              Active_Agent  
200              Active_Agent  
214              Active_Agent  
221              Active_Agent  
242              Active_Agent  
250              Active_Agent  
262                Background  
277                  Building  
294                  Building  
300                  Building  
329                Background  
338                  Building  
351                  Building  
356                  Building

In [7]:
data_Reduced.Collider_CategoricalN.unique()

array(['Background', 'TaskBuilding_Public', 'Building',
       'TaskBuilding_Residential', 'Active_Agent', 'Global_Landmark',
       'Active_Agent_Face', 'Passive_Agent', 'Passive_Agent_Face'],
      dtype=object)

In [8]:
import pandas as pd
import numpy as np
from math import log2

# Load the data
data = pd.read_csv("/Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_1.csv")
data['date_seconds'] = pd.to_datetime(data['timeStampDataPointEnd'], unit='s')

# Filter for the desired gaze events
data_Reduced = data[data['events'] == -2]

# Collider list
collider_list = [
    '56_Sa', '39_Sa', '19_Cma', '55_Sa', '25_Cma', '40_Sa', '41_Sa',
    '17_Cma', '47_Sa', '03_Cma', '13_Cma', '24_Cma', '01_Cma', '54_Sa',
    '15_Cma', '29_Sa', '04_Cma', '49_Sa', '30_Sa', '02_Cma', '51_Sa',
    '08_Cma', '28_Cma', '26_Cma', '44_Sa', '06_Cma', '53_Sa', '37_Sa',
    '32_Sa', '20_Cma', '16_Cma', '50_Sa', '34_Sa', '11_Cma', '38_Sa',
    '33_Sa', '12_Cma', '22_Cma', '42_Sa', '05_Cma', '23_Cma', '18_Cma',
    '27_Cma', '45_Sa', '43_Sa', '09_Cma', '31_Sa', '48_Sa', '10_Cma',
    '52_Sa', '07_Cma', '46_Sa', '35_Sa', '36_Sa', '21_Cma', '14_Cma'
]

# Filter and label rows with colliders
filtered_df = data_Reduced[data_Reduced['names'].isin(collider_list)].copy()
filtered_df['Occurrence_Order'] = filtered_df.groupby('names').cumcount() + 1

# Maintain a dictionary to track the last processed time for each collider
last_processed_time = {}

# Segment data by each occurrence of colliders
trials = []
for index, row in filtered_df.iterrows():
    collider_name = row['names']
    occurrence_time = row['date_seconds']

    # Check if this occurrence falls within the active window
    if (
        collider_name in last_processed_time
        and (occurrence_time - last_processed_time[collider_name]).total_seconds() <= 30
    ):
        # Skip this occurrence since it's within the 30-second window
        continue

    # Update the last processed time for this collider
    last_processed_time[collider_name] = occurrence_time

    # Constrain the 30-second window to the dataset bounds
    window_start = max(data_Reduced['date_seconds'].min(), occurrence_time)
    window_end = min(data_Reduced['date_seconds'].max(), occurrence_time + pd.Timedelta(seconds=30))

    # Extract the constrained window
    trial_segment = data_Reduced[
        (data_Reduced['date_seconds'] >= window_start) &
        (data_Reduced['date_seconds'] <= window_end)
    ].copy()

    if trial_segment.empty:
        print(f"No data found in window for {collider_name}. Skipping.")
        continue

    # Add trial-specific labels
    trial_segment['Collider_Name'] = collider_name
    trial_segment['Occurrence_Order'] = len(trials) + 1  # Increment trial count
    trial_segment['Trial_ID'] = f"{collider_name}_Trial_{len(trials) + 1}"
    trials.append(trial_segment)

# Combine all trials into a single DataFrame
if trials:
    trials_df = pd.concat(trials, ignore_index=True)
else:
    print("No trials found. Exiting.")
    trials_df = pd.DataFrame()  # Create an empty DataFrame for safety

# Continue with entropy calculation if trials_df is not empty
if not trials_df.empty:
    # Initialize list to store entropy results
    entropy_results = []

    # Function to calculate conditional entropy
    def calculate_transition_entropy(matrix, stationary_distribution):
        total_entropy = 0
        category_entropies = {}
        for i, row in matrix.iterrows():
            row_entropy = 0
            for p in row:
                if p > 0:
                    row_entropy -= p * log2(p)
            category_entropies[i] = row_entropy
            total_entropy += row_entropy * stationary_distribution.get(i, 0)
        return total_entropy, category_entropies

    # Calculate entropy for each Trial_ID
    for trial_id, trial_data in trials_df.groupby('Trial_ID'):
        trial_data.loc[:, "Mapped_Column_Collider_Categorical"] = trial_data["Collider_CategoricalN"].replace(mapping)
        gaze_sequence = trial_data['Mapped_Column_Collider_Categorical'].reset_index(drop=True)
        categories = gaze_sequence.unique()
        transition_matrix = pd.DataFrame(0, index=categories, columns=categories, dtype=float)

        # Build the transition matrix
        for i in range(len(gaze_sequence) - 1):
            current_category = gaze_sequence.iloc[i]
            next_category = gaze_sequence.iloc[i + 1]
            transition_matrix.loc[current_category, next_category] += 1

        # Normalize the transition matrix
        transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0).fillna(0)

        # Check if the transition matrix is valid
        if transition_matrix.sum().sum() == 0 or transition_matrix.isnull().values.any():
            print(f"Skipping Trial_ID {trial_id}: Transition matrix is invalid or empty.")
            continue

        # Calculate stationary distribution
        try:
            eigvals, eigvecs = np.linalg.eig(transition_matrix.T)
            stationary_distribution = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
            stationary_distribution /= stationary_distribution.sum()

            stationary_distribution_dict = {categories[i]: stationary_distribution[i] for i in range(len(categories))}
        except Exception as e:
            print(f"Error calculating stationary distribution for Trial_ID {trial_id}: {e}")
            stationary_distribution_dict = {category: 1 / len(categories) for category in categories}

        # Calculate entropy
        overall_transition_entropy, transition_entropy_per_category = calculate_transition_entropy(
            transition_matrix, stationary_distribution_dict
        )

        stationary_entropy_per_category = {
            category: (-stationary_distribution_dict[category] * log2(stationary_distribution_dict[category]))
            if stationary_distribution_dict[category] > 0 else 0
            for category in categories
        }

        num_categories = len(transition_matrix)
        normalized_overall_entropy = overall_transition_entropy / log2(num_categories) if num_categories > 1 else 0

        result = {
            'Trial_ID': trial_id,
            'Overall_Transition_Entropy': normalized_overall_entropy
        }

        for category in categories:
            result[f'Transition_Entropy_{category}'] = (
                transition_entropy_per_category.get(category, 0) / log2(num_categories)
            ) if num_categories > 1 else 0
            result[f'Stationary_Entropy_{category}'] = (
                stationary_entropy_per_category.get(category, 0) / log2(num_categories)
            ) if num_categories > 1 else 0

        entropy_results.append(result)

    # Convert results into a DataFrame
    entropy_df = pd.DataFrame(entropy_results)


In [9]:
entropy_df.head()

Trial_ID  Overall_Transition_Entropy  Transition_Entropy_Agent_Face  \
0  01_Cma_Trial_22                    0.570817                       0.605376   
1  03_Cma_Trial_16                    0.340640                       0.000000   
2  09_Cma_Trial_14                    0.289236                       0.266753   
3  09_Cma_Trial_24                    0.361897                            NaN   
4  09_Cma_Trial_26                    0.376103                            NaN   

   Stationary_Entropy_Agent_Face  Transition_Entropy_Agent_Body  \
0                       0.091280                       0.430677   
1                       0.027601                       0.619451   
2                       0.124505                       0.424315   
3                            NaN                       0.000000   
4                            NaN                       0.395488   

   Stationary_Entropy_Agent_Body  Transition_Entropy_Background  \
0                       0.062735                       0.689468   
1                       0.168669                       0.301320   
2                       0.068234                       0.204848   
3                       0.024192                       0.305478   
4                       0.047812                       0.342352   

   Stationary_Entropy_Background  Transition_Entropy_Task_Building  \
0                       0.228565                          0.440598   
1                       0.117293                               NaN   
2                       0.161853                          0.808547   
3                       0.139582                          0.539047   
4                       0.111824                          0.628421   

   Stationary_Entropy_Task_Building  Transition_Entropy_Building  \
0                          0.206797                     0.534424   
1                               NaN                     0.431560   
2                          0.130170                     0.379551   
3                          0.091174                     0.559766   
4                          0.103383                     0.511955   

   Stationary_Entropy_Building  Transition_Entropy_Global_Landmark  \
0                     0.227872                                 NaN   
1                     0.133281                                 NaN   
2                     0.159054                                 NaN   
3                     0.190264                            0.395488   
4                     0.153667                            0.000000   

   Stationary_Entropy_Global_Landmark  
0                                 NaN  
1                                 NaN  
2                                 NaN  
3                            0.064483  
4                            0.028284

In [ ]:
entropy_df.Overall_Transition_Entropy.describe()

In [ ]:
trials_df.head()